<a href="https://colab.research.google.com/github/comb0601/deepfake/blob/master/deepfake_1_1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mtcnn
!pip install facenet-pytorch
import argparse
import glob
import os
import glob
import cv2
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import skimage.measure
from skimage import measure
from facenet_pytorch import MTCNN
from PIL import Image

     |████████████████████████████████| 2.3MB 8.4MB/s 
     |████████████████████████████████| 1.9MB 9.1MB/s 


In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
#TEST_PATH = '/content/'
TRAIN_PATH = '/gdrive/My Drive/'

#metadata = '../input/deepfake-detection-challenge/train_sample_videos/metadata.json'

In [4]:
# load the filenames for train videos
train_fns = sorted(glob.glob(TRAIN_PATH + '*.mp4'))

# load the filenames for test videos
#test_fns = sorted(glob.glob(TEST_PATH + '*.mp4'))

print('There are {} samples in the train set.'.format(len(train_fns)))
#print('There are {} samples in the test set.'.format(len(test_fns)))

There are 2 samples in the train set.


In [5]:
train_fns

['/gdrive/My Drive/fake.mp4', '/gdrive/My Drive/real.mp4']

In [6]:
def get_frames(filename):
    '''
    Get all frames from the video
    INPUT:
        filename - video filename
    OUTPUT:
        frames - the array of video frames
    '''
    frames = []
    
    cap = cv2.VideoCapture(filename)
    
    #while(cap.isOpened()):
    while len(frames) < 301 :
        ret, frame = cap.read()
                
        if not ret:
            break;
            
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(image)

    cap.release()
    cv2.destroyAllWindows()
    return frames



In [ ]:
# 얼굴 감지
# Create face detector
for video in train_fns[:1] :
  frames = get_frames(video)
  mtcnn = MTCNN(margin=20, keep_all=False, post_process=False,device='cuda:0')  #keep_all : multiple faces in a single image,device='cuda:0'

  
  save_paths = [str(video)+f'image_{i}.jpg' for i in range(len(frames[:100]))]
  mtcnn(frames[:100], save_path=save_paths)

  save_paths = [str(video)+f'image_{i}.jpg' for i in range(len(frames[100:200]))]
  mtcnn(frames[100:200], save_path=save_paths)

  save_paths = [str(video)+f'image_{i}.jpg' for i in range(len(frames[200:300]))]
  mtcnn(frames[200:300], save_path=save_paths)

# 새 섹션

In [7]:

def edgeEntropy(image):
    edge = cv2.Canny(image, 50, 100)
    return skimage.measure.shannon_entropy(edge)

In [8]:
def edgeNoiseAnalysis(image, p1, p2):
    h, w = len(image), len(image[0])

    edge = cv2.Canny(image, p1, p2)

    base = cv2.getGaussianKernel(5, 5)
    kernel = np.outer(base, base.transpose())

    arr = cv2.filter2D(edge, -1, kernel)

    edgecount = 0
    arrcount = 0

    for i in range(w):
        for j in range(h):
            if arr[i][j] < 55:
                arr[i][j] = 0
            else:
                arr[i][j] = 255
                arrcount += 1

            if edge[i][j] == 255:
                edgecount += 1

    return (arrcount - edgecount) / edgecount * 100



In [9]:
def edgeDensityAnalysis(image):
    h, w = len(image), len(image[0])

    edge = cv2.Canny(image, 50, 100)

    total_slc = 0
    slc_include_edge = 0

    for i in range(w):
        for j in range(h):
            if edge[i][j] == 255:
                slc_include_edge += 1
            total_slc += 1

    return (slc_include_edge / total_slc) * 100


In [10]:
def totalVariance(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    mean = calcMatrixMean(image)
    sum_distance = 0
    size = 0
    for row in image:
        for pixel in row:
            sum_distance += ((pixel - mean) ** 2)
            size += 1

    return sum_distance / size


In [11]:
def totalEntropy(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return skimage.measure.shannon_entropy(image)


In [12]:
def dctCoefficient(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = np.float32(image) / 255.0
    dct = cv2.dct(image)
    image = np.uint8(dct * 255.0)

    return image

In [13]:
# 이미지 전체 밝기 값 평균
def calcMatrixMean(image):
    pixel_sum = 0
    size = 0
    for row in image:
        for pixel in row:
            pixel_sum += pixel
            size += 1

    return pixel_sum / size


In [14]:
# RGB 평균값 계산
def calcRGBCenter(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #RGB 형태로 변환
    image = image.reshape((image.shape[0] * image.shape[1], 3))
    k = 1
    clt = KMeans(n_clusters = k)
    clt.fit(image)

    return clt.cluster_centers_[0]



In [15]:
# HSV 평균값 계산
def calcHSVCenter(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV) #HSV 형태로 변환
    image = image.reshape((image.shape[0] * image.shape[1], 3))
    k = 1
    clt = KMeans(n_clusters = k)
    clt.fit(image)

    return clt.cluster_centers_[0]

In [16]:
 # 히스토그램 차이
def get_image_difference(image_1, image_2): 
    first_image_hist = cv2.calcHist([image_1], [0], None, [256], [0, 256])
    second_image_hist = cv2.calcHist([image_2], [0], None, [256], [0, 256])

    img_hist_diff = cv2.compareHist(first_image_hist, second_image_hist, cv2.HISTCMP_BHATTACHARYYA)
    #img_template_probability_match = cv2.matchTemplate(first_image_hist, second_image_hist, cv2.TM_CCOEFF_NORMED)[0][0]
    #img_template_diff = 1 - img_template_probability_match

    # taking only 10% of histogram diff, since it's less accurate than template method
    #commutative_image_diff = (img_hist_diff / 10) + img_template_diff
    return img_hist_diff

In [92]:
i1 = cv2.imread('/gdrive/MyDrive/fake.mp4image_0.jpg')
i2 = cv2.imread('/gdrive/MyDrive/fake.mp4image_0.jpg')
dif = totalVariance(i2) - totalVariance(i1)
dif

0.0

In [76]:
    #for videoname in train_fns : # 하나의 동영상 이름 가져오기
    videoname = train_fns[0]
    frames = []
    is_fake = 1
    if videoname.startswith("real"):
      is_fake = 0

    for i in range(100) :      # 가져온 동영상 얼굴 사진들로 프레임 생성
      frame = cv2.imread(videoname+'image_'+str(i)+'.jpg')
      frames.append(frame)

    resfile = open(videoname+"_result.csv", "a")
    resfile.write("mse,psnr,ssim,hist_diff,r_diff,g_diff,b_diff,h_diff,s_diff,v_diff,matrix_diff, dctcoefficient, totalentropy, totalvariance, edgedensity, edgenoise, edgeentropy, deepfake\n")

    for i in range(len(frames)-1) :
        img1 = frames[i] # 첫 프레임
        img2 = frames[i+1] #다음 프레임

        mse = np.sum((img1.astype("float") - img2.astype("float")) ** 2) #MSE
        mse /= float(img1.shape[0] * img1.shape[1])

        psnr = cv2.PSNR(img1, img2) # 프레임 간 PSNR값

        ssim = measure.compare_ssim(img1, img2,multichannel=True) # SSIM 멀티채널 오픈필요

        histogram_diff = get_image_difference(img1,img2) # 프레임간 히스토그램 차이

        rgb_diff = abs(calcRGBCenter(img2) - calcRGBCenter(img1))

        hsv_diff = abs(calcHSVCenter(img2) - calcHSVCenter(img1))

        matrix_diff = abs(calcMatrixMean(img2) - calcMatrixMean(img1))

        coefficient_diff = abs(dctCoefficient(img2) - dctCoefficient(img1))

        entropy_diff = abs(totalEntropy(img2) - totalEntropy(img1))

        variance_diff = abs(totalVariance(img2) - totalVariance(img1))

        edgedentsity_diff = abs(edgeDensityAnalysis(img2) - edgeDensityAnalysis(img1))


        edgenoise_diff = abs(auto_canny(img2) - auto_canny(img1))

     

        edgeentropy_diff = abs(edgeEntropy(img2) - edgeEntropy(img1))

        resfile.write(
            str(mse) + ',' + str(psnr) + ',' + str(ssim) + ',' + str(histogram_diff) + ',' +
            str(rgb_diff[0]) + ',' + str(rgb_diff[1]) + ',' + str(rgb_diff[2]) + ',' +
            str(hsv_diff[0]) + ',' + str(hsv_diff[1]) + ',' + str(hsv_diff[2]) + ',' +
            str(matrix_diff) + ',' + str(coefficient_diff) + ',' + str(entropy_diff) + ',' +
            str(variance_diff) + ',' + str(edgedentsity_diff) + ',' + str(edgenoise_diff) + ',' +
            str(edgeentropy_diff) + ',' + str(is_fake) + '\n'
        )



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: Use

In [77]:
    #for videoname in train_fns : # 하나의 동영상 이름 가져오기
    videoname = train_fns[1]
    frames = []
    is_fake = 1
    if videoname.startswith("real"):
      is_fake = 0

    for i in range(100) :      # 가져온 동영상 얼굴 사진들로 프레임 생성
      frame = cv2.imread(videoname+'image_'+str(i)+'.jpg')
      frames.append(frame)

    resfile = open(videoname+"_result.csv", "a")
    resfile.write("mse,psnr,ssim,hist_diff,r_diff,g_diff,b_diff,h_diff,s_diff,v_diff,matrix_diff, dctcoefficient, totalentropy, totalvariance, edgedensity, edgenoise, edgeentropy, deepfake\n")

    for i in range(len(frames)-1) :
        img1 = frames[i] # 첫 프레임
        img2 = frames[i+1] #다음 프레임

        mse = np.sum((img1.astype("float") - img2.astype("float")) ** 2) #MSE
        mse /= float(img1.shape[0] * img1.shape[1])

        psnr = cv2.PSNR(img1, img2) # 프레임 간 PSNR값

        ssim = measure.compare_ssim(img1, img2,multichannel=True) # SSIM 멀티채널 오픈필요

        histogram_diff = get_image_difference(img1,img2) # 프레임간 히스토그램 차이

        rgb_diff = abs(calcRGBCenter(img2) - calcRGBCenter(img1))

        hsv_diff = abs(calcHSVCenter(img2) - calcHSVCenter(img1))

        matrix_diff = abs(calcMatrixMean(img2) - calcMatrixMean(img1))

        coefficient_diff = abs(dctCoefficient(img2) - dctCoefficient(img1))

        entropy_diff = abs(totalEntropy(img2) - totalEntropy(img1))

        variance_diff = abs(totalVariance(img2) - totalVariance(img1))

        edgedentsity_diff = abs(edgeDensityAnalysis(img2) - edgeDensityAnalysis(img1))


        edgenoise_diff = abs(auto_canny(img2) - auto_canny(img1))

     

        edgeentropy_diff = abs(edgeEntropy(img2) - edgeEntropy(img1))

        resfile.write(
            str(mse) + ',' + str(psnr) + ',' + str(ssim) + ',' + str(histogram_diff) + ',' +
            str(rgb_diff[0]) + ',' + str(rgb_diff[1]) + ',' + str(rgb_diff[2]) + ',' +
            str(hsv_diff[0]) + ',' + str(hsv_diff[1]) + ',' + str(hsv_diff[2]) + ',' +
            str(matrix_diff) + ',' + str(coefficient_diff) + ',' + str(entropy_diff) + ',' +
            str(variance_diff) + ',' + str(edgedentsity_diff) + ',' + str(edgenoise_diff) + ',' +
            str(edgeentropy_diff) + ',' + str(is_fake) + '\n'
        )



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: Use

In [10]:
REAL = pd.read_csv('/gdrive/My Drive/real.mp4_result.csv')
FAKE = pd.read_csv('/gdrive/MyDrive/fake.mp4_result .csv')

FAKE.head()

,mse,psnr,ssim,hist_diff,r_diff,g_diff,b_diff,h_diff,s_diff,v_diff,deepfake
0,1258.759453,21.902589,0.666231,0.072090,3.521992,1.739141,1.697461,0.803945,1.024805,3.385352,1
1,373.429375,27.179931,0.861296,0.065709,0.889609,0.764102,0.758008,0.865742,0.016328,0.906914,1
2,1065.294844,22.627318,0.737670,0.063018,0.284531,0.319727,0.545898,0.540820,0.305312,0.369492,1
3,453.817109,26.333208,0.844062,0.062875,1.220859,0.306602,0.719883,0.007305,0.133672,1.112461,1
4,190.464531,30.103875,0.920935,0.055805,0.336328,0.143555,0.370664,0.716758,0.168477,0.347695,1


In [11]:
REAL.mean()

mse          817.637949
psnr          24.949479
ssim           0.781395
hist_diff      0.060466
r_diff         1.333380
g_diff         0.709534
b_diff         0.741168
h_diff         0.604429
s_diff         0.653163
v_diff         1.275013
deepfake       0.000000
dtype: float64

In [12]:
FAKE.mean()

mse          725.975178
psnr          31.756990
ssim           0.771314
hist_diff      0.060635
r_diff         1.824142
g_diff         1.116223
b_diff         1.048626
h_diff         0.643826
s_diff         0.678161
v_diff         1.796061
deepfake       1.000000
dtype: float64

In [13]:
FAKE.describe()

,mse,psnr,ssim,hist_diff,r_diff,g_diff,b_diff,h_diff,s_diff,v_diff,deepfake
count,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.0
mean,725.975178,31.756990,0.771314,0.060635,1.824142,1.116223,1.048626,0.643826,0.678161,1.796061,1.0
std,441.441478,47.620370,0.098711,0.012689,1.469976,0.852953,0.823303,0.476198,0.609355,1.440220,0.0
min,0.000000,18.821444,0.528466,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,414.422344,22.863322,0.698974,0.055525,0.510625,0.451191,0.388613,0.257715,0.238027,0.500430,1.0
50%,608.633477,25.058458,0.775875,0.060603,1.358477,0.815469,0.773281,0.582266,0.452813,1.381914,1.0
75%,1009.088496,26.728091,0.842160,0.066190,2.970137,1.641289,1.652520,0.894336,0.987988,2.907598,1.0
max,2558.923086,361.201999,1.000000,0.088132,5.371953,3.304961,3.323867,1.987500,2.386719,5.303281,1.0


In [14]:
REAL.describe()

,mse,psnr,ssim,hist_diff,r_diff,g_diff,b_diff,h_diff,s_diff,v_diff,deepfake
count,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.0
mean,817.637949,24.949479,0.781395,0.060466,1.333380,0.709534,0.741168,0.604429,0.653163,1.275013,0.0
std,611.862172,3.336766,0.106772,0.011246,1.043695,0.534190,0.565289,0.513660,0.524101,0.997353,0.0
min,92.639961,18.225337,0.537962,0.039971,0.011484,0.022891,0.007812,0.000469,0.018555,0.022773,0.0
25%,396.249141,22.476283,0.702624,0.051639,0.540566,0.271172,0.270664,0.200488,0.263691,0.433496,0.0
50%,628.837148,24.916634,0.795811,0.058880,1.049766,0.601797,0.604766,0.449180,0.508281,1.013945,0.0
75%,1103.008594,26.922839,0.862608,0.067528,2.043203,1.035371,1.110098,0.921504,0.938438,1.982578,0.0
max,2935.404414,33.234033,0.948391,0.089393,3.807773,2.225273,2.294453,1.985430,3.008945,3.570234,0.0


In [15]:
FAKE.corr()

,mse,psnr,ssim,hist_diff,r_diff,g_diff,b_diff,h_diff,s_diff,v_diff,deepfake
mse,1.000000,-0.287749,-0.924003,0.428628,0.659170,0.606441,0.616534,0.192473,0.529060,0.659191,NaN
psnr,-0.287749,1.000000,0.383414,-0.702470,-0.214883,-0.223042,-0.217275,-0.202879,-0.187284,-0.215787,NaN
ssim,-0.924003,0.383414,1.000000,-0.446745,-0.676292,-0.644046,-0.638227,-0.192052,-0.488555,-0.676379,NaN
hist_diff,0.428628,-0.702470,-0.446745,1.000000,0.422748,0.356783,0.379767,0.283659,0.341890,0.419881,NaN
r_diff,0.659170,-0.214883,-0.676292,0.422748,1.000000,0.928961,0.922619,0.216734,0.534980,0.999477,NaN
g_diff,0.606441,-0.223042,-0.644046,0.356783,0.928961,1.000000,0.931885,0.187022,0.401485,0.932962,NaN
b_diff,0.616534,-0.217275,-0.638227,0.379767,0.922619,0.931885,1.000000,0.149629,0.284185,0.925106,NaN
h_diff,0.192473,-0.202879,-0.192052,0.283659,0.216734,0.187022,0.149629,1.000000,0.238880,0.214940,NaN
s_diff,0.529060,-0.187284,-0.488555,0.341890,0.534980,0.401485,0.284185,0.238880,1.000000,0.534723,NaN
v_diff,0.659191,-0.215787,-0.676379,0.419881,0.999477,0.932962,0.925106,0.214940,0.534723,1.000000,NaN


In [16]:
REAL.corr()

,mse,psnr,ssim,hist_diff,r_diff,g_diff,b_diff,h_diff,s_diff,v_diff,deepfake
mse,1.000000,-0.906393,-0.906311,0.726092,0.619384,0.431131,0.283143,0.186248,0.534627,0.592033,NaN
psnr,-0.906393,1.000000,0.921746,-0.695548,-0.611398,-0.491624,-0.361729,-0.167615,-0.493511,-0.589803,NaN
ssim,-0.906311,0.921746,1.000000,-0.587211,-0.616308,-0.510269,-0.374980,-0.056115,-0.509979,-0.601188,NaN
hist_diff,0.726092,-0.695548,-0.587211,1.000000,0.478277,0.400282,0.315062,0.275239,0.454172,0.469141,NaN
r_diff,0.619384,-0.611398,-0.616308,0.478277,1.000000,0.782266,0.740178,0.054346,0.386110,0.993179,NaN
g_diff,0.431131,-0.491624,-0.510269,0.400282,0.782266,1.000000,0.836889,0.032049,0.234234,0.822642,NaN
b_diff,0.283143,-0.361729,-0.374980,0.315062,0.740178,0.836889,1.000000,-0.012656,0.043839,0.774187,NaN
h_diff,0.186248,-0.167615,-0.056115,0.275239,0.054346,0.032049,-0.012656,1.000000,0.174131,0.031443,NaN
s_diff,0.534627,-0.493511,-0.509979,0.454172,0.386110,0.234234,0.043839,0.174131,1.000000,0.372738,NaN
v_diff,0.592033,-0.589803,-0.601188,0.469141,0.993179,0.822642,0.774187,0.031443,0.372738,1.000000,NaN


In [46]:
def auto_canny(image, sigma=0.33):
	# compute the median of the single channel pixel intensities
	v = np.median(image)
	# apply automatic Canny edge detection using the computed median
	lower = int(max(0, (1.0 - sigma) * v))
	upper = int(min(255, (1.0 + sigma) * v))
	edged = cv2.Canny(image, lower, upper)
	# return the edged image
	return edged

In [68]:
im = cv2.imread('/gdrive/MyDrive/fake.mp4image_0.jpg')
auto_canny(im[0][0][0])

TypeError: ignored